In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import os

# Configurações
model_name = "BAAI/bge-m3"
output_folder = 'vectorstore'
existing_db_path = 'vectorstore/db_faiss_all_pdfs'


def load_and_process_pdfs(pdf_folder):
    documents = []
    for filename in os.listdir(pdf_folder):
        if filename.endswith('.pdf'):
            file_path = os.path.join(pdf_folder, filename)
            loader = PyPDFLoader(file_path)
            documents.extend(loader.load())
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    split_docs = text_splitter.split_documents(documents)
    return split_docs


def get_vectorstore(pdf_folder):
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    if os.path.exists(existing_db_path):
        return FAISS.load_local(existing_db_path, embeddings, allow_dangerous_deserialization=True)
    else:
        docs = load_and_process_pdfs(pdf_folder)
        vectorstore = FAISS.from_documents(docs, embeddings)
        vectorstore.save_local(existing_db_path)
        return vectorstore


vectorstore = get_vectorstore('pdf_ppg')
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={'k': 3, 'fetch_k': 5})

c:\Users\Tekla\anaconda3\envs\LLM\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
def model_ollama(model="llama3.2", temperature=0.1):
    return ChatOllama(model=model, temperature=temperature)

llm = model_ollama()

In [4]:
context_q_system_prompt = """
Dado o histórico de chat e a pergunta de acompanhamento, que pode fazer referência ao contexto no histórico do chat, 
formule uma pergunta independente que possa ser entendida sem o histórico do chat. 
NÃO responda à pergunta, apenas reformule-a se necessário e, caso contrário, retorne-a como está.
"""

qa_prompt_template = """
Você é um assistente virtual especializado em analisar e responder perguntas sobre documentos juridicos.
Use os seguintes trechos de contexto recuperado para responder à pergunta.
Se você não souber a resposta, diga honestamente que não sabe. Mantenha a resposta concisa e focada.
Se for solicitado a listar as referências dos artigos, leias, informações específicas do documento, faça-o de forma estruturada e clara.
Responda em português.

Pergunta: {input}

Contexto: {context}

Resposta:
"""

context_q_prompt = ChatPromptTemplate.from_messages([
    ("system", context_q_system_prompt),
    ("human", "Pergunta: {input}")
])

history_aware_retriever = create_history_aware_retriever(llm=llm, retriever=retriever, prompt=context_q_prompt)

qa_prompt = PromptTemplate.from_template(qa_prompt_template)
qa_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

# Função para fazer perguntas
def ask_question(question, chat_history=''): 
    result = rag_chain.invoke({"input": question, "chat_history": chat_history})
    return result['answer']

Pagina 390 do documento

In [5]:
user_query = 'De acordo com o laudo foi comprovado doença ocupacional?'
response = ask_question(user_query)
print(response)

Não sabe se foi comprovado doença ocupacional de acordo com o laudo.


Pagina 424 do documento

In [6]:
user_query = """Extrair alguns pontos relevantes da sentença. Sobre as horas extras qual foi a sentença do Juiz?"""
response = ask_question(user_query)
print(response)

A sentença do Juiz menciona que o autor reclamou o pagamento de horas extras excedentes à oitava hora diária e 44ª hora semanal, mas o Juiz a quo afastou a exceção prevista no art. 62 da CLT, considerando que a jornada de trabalho informada na inicial (segunda à quinta-feira, das 8h às 12h e 13h às 20h e, nas sextas-feiras, das 08h às 17h) não é a mesma que a jornada efetivamente cumprida pelo autor.

Portanto, o Juiz julgou parcialmente procedente o pedido de pagamento de horas extras, mas apenas com base na jornada fixada acima (08h às 20h, com 1 hora de intervalo), e não com base nas horas extras excedentes à oitava hora diária e 44ª hora semanal.

Em resumo, a sentença do Juiz afasta a exceção prevista no art. 62 da CLT e julga parcialmente procedente o pedido de pagamento de horas extras com base na jornada fixada acima.
